LIBRARY

In [2]:
import requests
import pandas as pd
import requests
import time
import urllib
from bs4 import BeautifulSoup
import json
from functools import *

Crawling_Product_ID

In [3]:
def get_product_catagories(url):
    def find_value_of_key(value, key):
        val = None
        if isinstance(value,dict):
            if key in value:
                return value[key]
            for i in value.items():
                val = find_value_of_key(i[1], key)
                if val is not None:
                    return val
        if isinstance(value,list):
            for i in value:
                val = find_value_of_key(i, key)
        return val
    http_headers = {
        'User-Agent':  'PostmanRUntime/7.29.2',
    }
    response = requests.get(url=url, headers=http_headers)
    parser = BeautifulSoup(response.text, 'html.parser', )
    product_box = parser.findAll('script')
    product_catagories = []
    for product in product_box:
        try:
            item = json.loads(str(product.contents[0]))
        except:
            item = {}
        product_val = find_value_of_key(item,"breadcrumbs")
        if product_val is not None:
            [product_catagories.append(i) for i in product_val if i["category_id"] != 0] 
    return product_catagories

get_product_catagories('https://tiki.vn/dien-thoai-samsung-galaxy-z-flip-4-8gb-128gb-hang-chinh-hang-p194034217.html?itm_campaign=CTP_YPD_TKA_PLA_UNK_ALL_UNK_UNK_UNK_UNK_X.163088_Y.1744616_Z.3253348_CN.Product-Ads-B4Q4&itm_medium=CPC&itm_source=tiki-ads&spid=194034225')


[{'url': '/dien-thoai-may-tinh-bang/c1789',
  'name': 'Điện Thoại - Máy Tính Bảng',
  'category_id': 1789},
 {'url': '/dien-thoai-smartphone/c1795',
  'name': 'Điện thoại Smartphone',
  'category_id': 1795}]

#GET ID OF PRODUCT

In [4]:
def get_max_page_number(name:str, categories_id:int):
    http_headers = {
    'User-Agent':  'PostmanRUntime/7.29.2',
    }
    url =f'https://tiki.vn/{name}/c{categories_id}'
    response = None
    while(response is None):
        response = requests.get(url=url, headers=http_headers)
    parser = BeautifulSoup(response.text, 'html.parser', )

    # find all 
    div_tag = parser.find("div", {"data-view-id":"product_list_pagination_container"})   
    ul_tag = div_tag.find('ul')
    li_tag = ul_tag.findAll('li')
    def find_max_tag(a,b):
        if isinstance(a,int):
            a_num = a
        else:
            a_tag = a.find('a')
            a_num = a_tag.get('data-view-label') if a_tag.get('data-view-label') is not None else 0
        b = b.find('a')
        b_num = b.get('data-view-label') if b.get('data-view-label') is not None else 0
        return max(int(a_num), int(b_num))
    
    return reduce(find_max_tag,li_tag)





get_max_page_number("Điện thoại Smartphone", 1795)

12

crawling_catagories_id

In [113]:
def crawling_product_id(name:str, categories_id:int, page_number:int):
    http_headers = {
        'User-Agent':  'PostmanRUntime/7.29.2',
    }
    product_list = set()
    for i in range(page_number):
        url =f'https://tiki.vn/{name}/c{categories_id}?page{i}'
        response = None
        while(response is None):
            response = requests.get(url=url, headers=http_headers)
        parser = BeautifulSoup(response.text, 'html.parser', )
    
        # find all 
        product_box = parser.findAll('a')
        if (len(product_box) == 0):
            pass
        for product in product_box:
            data = product.get("data-view-content")
            if data is not None:
                item = json.loads(data)
                id = item['click_data'].get('id')
                if id is not None:
                    product_list.add(id)
    return list(product_list)
crawling_product_id("Điện thoại Smartphone", 1795,12)

[197553025,
 177746939,
 155536275,
 184051220,
 197216279,
 177343384,
 170581659,
 201631518,
 197670818,
 170580388,
 194034217,
 120295859,
 114092342,
 127372598,
 187266104,
 203764665,
 88100671,
 197214015,
 184059211,
 164923596,
 197214029,
 168282061,
 194037967,
 164937168,
 201548628,
 160801751,
 203864283,
 113569117,
 184036446,
 158714974,
 166034912,
 101615454,
 189658085,
 187254759,
 170497385,
 206131571,
 142545270,
 175845367,
 132068858,
 114766715]

In [6]:
http_headers = {
    'User-Agent':  'PostmanRUntime/7.29.2',
}
response = requests.get(url="https://tiki.vn/dien-thoai-samsung-galaxy-z-flip-4-8gb-128gb-hang-chinh-hang-p194034217.html?itm_campaign=CTP_YPD_TKA_PLA_UNK_ALL_UNK_UNK_UNK_UNK_X.163088_Y.1744616_Z.3253348_CN.Product-Ads-B4Q4&itm_medium=CPC&itm_source=tiki-ads&spid=194034225", headers=http_headers)
parser = BeautifulSoup(response.text, 'html.parser' )
with open('data.txt', 'w') as file:
    data = file.write(response.content.decode())
print(data)

285782
